# Estamos em Produção

### Verificar a escala das falhas utilizando o Gradiente Boost Regressor

#### 1 - Carregar o modelo treinado
#### 2 - Carregar os dados a serem inferidos
#### 3 - Fazer as inferências
#### 4 - FIM

In [ ]:
# Carregando modelo

import pickle
model = pickle.load(open('model_GBT.sav', 'rb'))

In [ ]:
import pandas as pd

df = pd.read_excel('O_G_Equipment_Data.xlsx')
df.head(2)

,Cycle,Preset_1,Preset_2,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Fail
0,1,3,6,44.235186,47.657254,46.441769,64.820327,66.454520,44.483250,False
1,2,2,4,60.807234,63.172076,62.005951,80.714431,81.246405,60.228715,False


In [ ]:
labels = df['Fail'].astype('category').cat.categories.tolist()
replace_Fail = {'Fail':{False: 10, True: 100}}

print(replace_Fail)

{'Fail': {False: 10, True: 100}}


In [ ]:
df.replace(replace_Fail, inplace=True)

In [ ]:
''' Trecho de código utilizado para realizar inferências futuras '''

df_WF = df.copy()
df_QF = df.copy()

## CATEGORIZANDO AS FALHAS, CRIANDO UM GRAU DE INTENSIDADE E/OU CRITICIDADE PARA A FALHA DETECTADA.

In [ ]:
df_WF.drop(df_WF[(df_WF['Fail'] == 10)].index, inplace=True)
df_WF.head(2)

,Cycle,Preset_1,Preset_2,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Fail
12,13,1,1,79.806509,103.129738,120.753103,134.500905,115.873197,76.562856,100
156,157,3,1,91.695332,85.345796,99.890670,95.328429,124.970301,84.377355,100


In [ ]:
df_2 = df_WF.drop(columns={'Cycle', 'Preset_1','Preset_2','Fail'})
df_2.head(2)

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency
12,79.806509,103.129738,120.753103,134.500905,115.873197,76.562856
156,91.695332,85.345796,99.890670,95.328429,124.970301,84.377355


In [ ]:
# Inferência

inferences = model.predict(df_2)
df_2['Resultado'] = inferences
df_2.head(2)

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
12,79.806509,103.129738,120.753103,134.500905,115.873197,76.562856,1.827586
156,91.695332,85.345796,99.890670,95.328429,124.970301,84.377355,1.012712


In [ ]:
df_f = pd.DataFrame(df_2.Resultado.value_counts().sort_index())
df_f = df_f.reset_index()
df_f = df_f.rename({'index': 'Fail_Category'}, axis=1)
df_f

,Fail_Category,Resultado
0,1.012712,8
1,1.174242,30
2,1.827586,28


## ESTUDO ESTATÍTICO DAS CATEGORIAS.

#### Após a inferência realizada pelo algoritmo, criando 3 categorias diferentes, é o momento de se realizar um estudo estatísco a partir do momento em qua se divide a base original, na quantidade de novas categorias encontradas e entender qual ou quais são os atributos que se tornam predominantes em cada uma delas.


## Visuzalização das categorias

#### Como o algoritmo categorizou as falhas em três grupos distintos, é interessante que se utilize de um recurso gráfico visual, para entender quais foram os critérios apontados pela IA para separar as categorias. Logo, a seguir serão carregados trechos de códigos pertinentes a análise visual.

In [ ]:
!pip install -U datapane
!pip install chart_studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datapane as dp
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import chart_studio
import os
plt.rcParams.update({'font.size': 22})
%matplotlib inline

In [ ]:
# Realizar o login do datapane a partir da API disponibilizada pelo aplicação
!datapane login --token=715ffba6fea8179028c770596c699c494c0df0a6

Connected successfully to https://cloud.datapane.com as cristovaosouzajunior@gmail.com


### Exite uma reta que representa "união" de todos os pontos aleatórios que indicam uma falha, esse primeiro dataset, representam pontos demarcados em 1.012.

In [ ]:
df1 = df_2[df_2['Resultado'] <= 1.174242]


In [ ]:
df1.head()

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
156,91.695332,85.345796,99.890670,95.328429,124.970301,84.377355,1.012712
158,88.768766,99.228691,89.109514,124.934221,84.306467,88.218162,1.012712
264,100.986610,75.277544,114.859306,90.512733,115.104346,85.794190,1.012712
414,78.862065,101.454563,200.749598,50.481820,100.000843,50.456673,1.012712
415,76.867426,50.822002,101.404636,100.327893,200.242674,50.415427,1.012712


In [ ]:
len(df1)

8

In [ ]:
df1.describe()

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
count,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,81.049386,86.272126,126.707908,92.608872,128.589666,67.223719,1.012712
std,14.778051,18.936029,37.665475,22.028846,36.838890,18.661608,0.000000
min,50.469522,50.822002,89.109514,50.481820,84.306467,48.923777,1.012712
25%,78.363405,74.737889,101.026145,86.641511,104.114945,50.271920,1.012712
50%,80.372684,91.790609,110.763927,97.334904,120.037324,65.109720,1.012712
75%,89.500408,99.785159,149.943115,101.475083,149.070837,84.731564,1.012712
max,100.986610,106.694072,200.749598,124.934221,200.242674,88.218162,1.012712


In [ ]:
import plotly.express as px 

# On the YAML of code "back-end", modify the section yaml.load(f) to yaml.full_load(f)

# Trecho responsável por identificar e definir quais variáveis estarão contidas no eixo Y
equip_features = ['VibrationX', 'VibrationZ','Fail']

# desenvolvimento da visualização, organização e visualização
fig = px.line(df, x='Cycle' ,y=equip_features)
fig.show()
# trecho responsável por criar e publicar o relatório no datapane
dp.Report(
    dp.Plot(fig),
    dp.DataTable(df)
).upload(name="Failure over Cycle")

Uploading app and associated data - *please wait...*
App successfully uploaded. View and share your app at at https://cloud.datapane.com/reports/87NVEe7/failure-over-cycle/.


In [ ]:
df1.corr().style.format("{:.2}").background_gradient(cmap=("coolwarm"),axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2813: RuntimeWarning:

All-NaN slice encountered

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2814: RuntimeWarning:

All-NaN slice encountered



,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
Temperature,1.0,-0.2,-0.42,0.042,-0.39,0.7,nan
Pressure,-0.2,1.0,0.26,0.031,-0.8,0.28,nan
VibrationX,-0.42,0.26,1.0,-0.87,-0.058,-0.69,nan
VibrationY,0.042,0.031,-0.87,1.0,-0.031,0.56,nan
VibrationZ,-0.39,-0.8,-0.058,-0.031,1.0,-0.62,nan
Frequency,0.7,0.28,-0.69,0.56,-0.62,1.0,nan
Resultado,nan,nan,nan,nan,nan,nan,nan


### Exite uma reta que representa "união" de todos os pontos aleatórios que indicam uma falha, esse primeiro dataset, representam pontos demarcados em 1.17.

In [ ]:
df2 = df_2[(df_2['Resultado'] >= 1.012712) & (df_2['Resultado'] <= 1.827586	)]

In [ ]:
len(df2)

30

In [ ]:
df2.describe()

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,96.122005,111.013833,100.187307,103.337933,101.833745,105.427321,1.174242
std,22.637361,25.383913,23.282457,16.893025,24.860113,17.350454,0.000000
min,48.171371,69.923414,69.809370,71.189361,70.677784,89.329631,1.174242
25%,83.346896,90.876424,85.221098,91.137184,83.417847,94.104651,1.174242
50%,95.061930,102.377393,94.653453,102.675393,97.541838,100.298022,1.174242
75%,106.558814,124.219654,120.169610,119.624014,116.702667,111.139248,1.174242
max,150.880308,168.488300,161.188743,125.699313,160.239558,178.090303,1.174242


In [ ]:
# Trecho responsável por identificar e definir quais variáveis estarão contidas no eixo Y
equip_features = ['Temperature','Frequency','Fail']

# desenvolvimento da visualização, organização e visualização
fig = px.line(df, x='Cycle', y=equip_features)
fig.show()

# trecho responsável por criar e publicar o relatório no datapane
dp.Report(
    dp.Plot(fig),
    dp.DataTable(df)
).upload(name="Failure over Cycle")

Uploading app and associated data - *please wait...*
App successfully uploaded. View and share your app at at https://cloud.datapane.com/reports/87NVEe7/failure-over-cycle/.


In [ ]:
df2.corr().style.format("{:.2}").background_gradient(cmap=("coolwarm"),axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2813: RuntimeWarning:

All-NaN slice encountered

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2814: RuntimeWarning:

All-NaN slice encountered



,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
Temperature,1.0,0.38,0.092,0.17,0.022,-0.46,nan
Pressure,0.38,1.0,-0.08,0.25,-0.33,-0.27,nan
VibrationX,0.092,-0.08,1.0,-0.55,0.5,-0.37,nan
VibrationY,0.17,0.25,-0.55,1.0,-0.39,0.088,nan
VibrationZ,0.022,-0.33,0.5,-0.39,1.0,-0.21,nan
Frequency,-0.46,-0.27,-0.37,0.088,-0.21,1.0,nan
Resultado,nan,nan,nan,nan,nan,nan,nan


### Exite uma reta que representa "união" de todos os pontos aleatórios que indicam uma falha, esse primeiro dataset, representam pontos demarcados em 1.82.

In [ ]:
df3 = df_2[df_2['Resultado'] >= 1.827586]

In [ ]:
len(df3)

28

In [ ]:
df3.describe()

,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,2.800000e+01
mean,91.818107,130.831988,92.562574,151.526697,104.017197,103.108278,1.827586e+00
std,21.702156,29.499941,26.143153,17.723938,27.357507,22.238750,6.783575e-16
min,48.091962,90.183752,20.120467,129.691729,20.522642,70.853353,1.827586e+00
25%,79.719557,109.010972,86.284479,135.269307,90.327831,90.023790,1.827586e+00
50%,94.541794,125.955102,93.152206,149.658643,99.949298,98.920267,1.827586e+00
75%,105.380226,149.516999,103.294525,160.766934,120.542852,111.156682,1.827586e+00
max,127.670754,189.995681,158.556452,193.569947,157.083613,150.558790,1.827586e+00


In [ ]:
# Trecho responsável por identificar e definir quais variáveis estarão contidas no eixo Y
equip_features = ['Pressure','VibrationY','Fail']

# desenvolvimento da visualização, organização e visualização
fig = px.line(df, x='Cycle', y=equip_features)
fig.show()

# trecho responsável por criar e publicar o relatório no datapane
dp.Report(
    dp.Plot(fig),
    dp.DataTable(df)
).upload(name="Failure over Cycle")

Uploading app and associated data - *please wait...*
App successfully uploaded. View and share your app at at https://cloud.datapane.com/reports/87NVEe7/failure-over-cycle/.


In [ ]:
df3.corr().style.format("{:.2}").background_gradient(cmap=("coolwarm"),axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2813: RuntimeWarning:

All-NaN slice encountered

/usr/local/lib/python3.7/dist-packages/pandas/io/formats/style.py:2814: RuntimeWarning:

All-NaN slice encountered



,Temperature,Pressure,VibrationX,VibrationY,VibrationZ,Frequency,Resultado
Temperature,1.0,-0.0045,0.33,-0.082,0.24,-0.28,nan
Pressure,-0.0045,1.0,-0.11,0.6,-0.29,0.36,nan
VibrationX,0.33,-0.11,1.0,-0.38,0.38,-0.47,nan
VibrationY,-0.082,0.6,-0.38,1.0,0.04,0.39,nan
VibrationZ,0.24,-0.29,0.38,0.04,1.0,-0.51,nan
Frequency,-0.28,0.36,-0.47,0.39,-0.51,1.0,nan
Resultado,nan,nan,nan,nan,nan,nan,nan
